In [1]:
import cv2 as cv
import numpy as np
import HandTrackingModule as htm
import math

In [2]:
import subprocess

# تنظیم سطح صدا (به عنوان مثال 50 درصد)
def set_volume(volume_level):
    script = f"set volume output volume {volume_level}"
    subprocess.run(["osascript", "-e", script])

# دریافت سطح فعلی صدا
def get_volume():
    script = "output volume of (get volume settings)"
    result = subprocess.run(["osascript", "-e", script], capture_output=True, text=True)
    return int(result.stdout.strip())

# تست کد
set_volume(30)  # تنظیم صدا به 50 درصد
current_volume = get_volume()
print(f"Current volume level: {current_volume}")


Current volume level: 30


In [3]:
print(get_volume())

30


In [4]:
webcam = cv.VideoCapture(0)
wCam , hCam = 640 , 480
webcam.set(3,wCam)
webcam.set(4,hCam)

detector = htm.handdetectors(detectionCon=0.7)

vol = 0

while True:
    ok,frame = webcam.read()
    #frame = cv.flip(frame,1)

    if not ok:
        break

    tm = cv.TickMeter()
    tm.start()

    hand_detected = detector.findhands(frame)
    lm_list = detector.findposition(frame,False)


    if len(lm_list) != 0 :
        x1,y1 = lm_list[4][1],lm_list[4][2]
        x2,y2 = lm_list[8][1],lm_list[8][2]
        cx,cy = (x1+x2) // 2 , (y1+y2) // 2

        cv.circle(hand_detected,(x1,y1),10,(255,0,255),-1)
        cv.circle(hand_detected,(x2,y2),10,(255,0,255),-1)
        cv.circle(hand_detected,(cx,cy),10,(255,0,255),-1)

        cv.line(hand_detected,(x1,y1),(x2,y2),(255,0,255),3)

        lenthg = math.hypot(x2 - x1,y2 - y1)
        vol = np.interp(lenthg,[50,270],[0,100])
        vol_bar = np.interp(lenthg,[50,270],[470,170])
        set_volume(vol)
        cv.rectangle(hand_detected,(30,int(vol_bar)),(70,470),(128,128,128),-1)

    cv.rectangle(hand_detected,(30,170),(70,470),(128,128,128),3)
    
    tm.stop()

    cv.putText(hand_detected,f'FPS : {str(int(round(tm.getFPS())))}',(30,30),cv.FONT_HERSHEY_PLAIN,2,(255,0,0),2)
    cv.putText(hand_detected,f'vol : {str(int(vol))}',(30,150),cv.FONT_HERSHEY_COMPLEX,1,(0,0,0),2)

    
    cv.imshow('frame',hand_detected)

    if cv.waitKey(1) & 0xff == ord('q'):
        break

cv.destroyAllWindows()

webcam.release()
 

I0000 00:00:1725307445.528393    9665 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1725307445.535604   10712 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725307445.542287   10712 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2024-09-02 23:34:05.643 Python[1480:9665] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
/Users/matin/Downloads/quera/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be r

: 